# Libraries

In [16]:
import pandas as pd
import requests
from shapely import wkt
import numpy as np

# Helpful functions

In [2]:
""" def calculate_area(wkt_polygon):
    polygon = wkt.loads(wkt_polygon)
    return polygon.area
     
      
    This function is deprecated"""


' def calculate_area(wkt_polygon):\n    polygon = wkt.loads(wkt_polygon)\n    return polygon.area\n     \n      \n    This function is deprecated'

In [3]:
API_KEY = '00f95217d3b04a0c9e1af341b4e1608a'

def get_coordinates(county, name, state="Florida"):
    place_name = f"{name}, {county} County, {state}"
    url = f"https://api.opencagedata.com/geocode/v1/json?q={place_name}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data['results']:
        location = data['results'][0]['geometry']
        return (location['lat'], location['lng'])
    else:
        return (None, None)


This function is gonna get our coordenates using the OpenCage Geocoding API using a key provided by one of the data scientists

In [ ]:
def get_coordinates(county, name, state="Florida"):
    place_name = f"{name}, {county} County, {state}"
    url = f"https://api.opencagedata.com/geocode/v1/json?q={place_name}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data['results']:
        location = data['results'][0]['geometry']
        return (location['lat'], location['lng'])
    else:
        return (None, None)

# Loading the data

In [4]:
ds = pd.read_csv('florida-beach-names.csv')
ds

,WKT,COUNTY,NAME,created_user,created_date,last_edited_user,last_edited_date
0,"POLYGON Z ((-9698711.156 3546590.3287 0,-96987...",ESCAMBIA,UNSURVEYED,NaN,NaN,NaN,NaN
1,"POLYGON Z ((-9061671.5384 3555608.0978 0,-9061...",DUVAL,HANNA PARK,NaN,NaN,NaN,NaN
2,"POLYGON Z ((-9054509.0537 3514807.6314 0,-9054...",ST JOHNS,GUANA RIVER SP,NaN,NaN,NaN,NaN
3,"POLYGON Z ((-9668169.2045 3552697.6667 0,-9668...",ESCAMBIA,UNSURVEYED,NaN,NaN,NaN,NaN
4,"POLYGON Z ((-9597884.3653 3547578.4878 0,-9597...",WALTON,WALTON COUNTY BCHS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
297,"POLYGON Z ((-9039717.3304 2937286.6217 0,-9039...",MONROE,ENP (HIGHLAND BEAC,NaN,NaN,NaN,NaN
298,"POLYGON Z ((-9039593.2653 2937286.6434 0,-9039...",MONROE,ENP (HIGHLAND BEAC,NaN,NaN,NaN,NaN
299,"POLYGON Z ((-9015596.0256 2844170.7527 0,-9015...",MONROE,LITTLE CRAWL KEY,NaN,NaN,NaN,NaN
300,"POLYGON Z ((-9016208.8296 2843020.5594 0,-9016...",MONROE,FAT DEER KEY,NaN,NaN,NaN,NaN


# Looking at the data

In [5]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   WKT               302 non-null    object 
 1   COUNTY            299 non-null    object 
 2   NAME              299 non-null    object 
 3   created_user      0 non-null      float64
 4   created_date      0 non-null      float64
 5   last_edited_user  0 non-null      float64
 6   last_edited_date  0 non-null      float64
dtypes: float64(4), object(3)
memory usage: 16.6+ KB


In [6]:
ds[ds['NAME'].isna()]

,WKT,COUNTY,NAME,created_user,created_date,last_edited_user,last_edited_date
27,"POLYGON Z ((-8926217.4063 2933624.3263 0,-8926...",NaN,NaN,NaN,NaN,NaN,NaN
28,"POLYGON Z ((-8927112.0082 2930542.2383 0,-8927...",NaN,NaN,NaN,NaN,NaN,NaN
153,"POLYGON Z ((-9213069.3505 3199083.0432 0,-9212...",NaN,NaN,NaN,NaN,NaN,NaN


# Data preprocessing

First thing thats gonna happen is we gonna get rid of the empty columns and the 3 null rows since non of that is gonna give any useful data

In [7]:
# We make a copy of the data to work on that
df = ds.copy()

In [8]:
df.drop(columns=['created_user', 'created_date', 'last_edited_user', 'last_edited_date'], inplace = True)

In [9]:
df

,WKT,COUNTY,NAME
0,"POLYGON Z ((-9698711.156 3546590.3287 0,-96987...",ESCAMBIA,UNSURVEYED
1,"POLYGON Z ((-9061671.5384 3555608.0978 0,-9061...",DUVAL,HANNA PARK
2,"POLYGON Z ((-9054509.0537 3514807.6314 0,-9054...",ST JOHNS,GUANA RIVER SP
3,"POLYGON Z ((-9668169.2045 3552697.6667 0,-9668...",ESCAMBIA,UNSURVEYED
4,"POLYGON Z ((-9597884.3653 3547578.4878 0,-9597...",WALTON,WALTON COUNTY BCHS
...,...,...,...
297,"POLYGON Z ((-9039717.3304 2937286.6217 0,-9039...",MONROE,ENP (HIGHLAND BEAC
298,"POLYGON Z ((-9039593.2653 2937286.6434 0,-9039...",MONROE,ENP (HIGHLAND BEAC
299,"POLYGON Z ((-9015596.0256 2844170.7527 0,-9015...",MONROE,LITTLE CRAWL KEY
300,"POLYGON Z ((-9016208.8296 2843020.5594 0,-9016...",MONROE,FAT DEER KEY


In [10]:
df.dropna(inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 301
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   WKT     299 non-null    object
 1   COUNTY  299 non-null    object
 2   NAME    299 non-null    object
dtypes: object(3)
memory usage: 9.3+ KB


Ok we dont have any nan rows in our data

## Feature engineering

Now based on our info of the beaches we are gonna find their approximate latitude and longitude

In [12]:
df['latitude'] = None
df['longitude'] = None

We initialize the two empty columns holding the coordenates

In [13]:
for index, row in df.iterrows():
    county = row['COUNTY']
    name = row['NAME']
    coordinates = get_coordinates(county, name)
    df.at[index, 'latitude'] = coordinates[0]
    df.at[index, 'longitude'] = coordinates[1]


In [14]:
df

,WKT,COUNTY,NAME,latitude,longitude
0,"POLYGON Z ((-9698711.156 3546590.3287 0,-96987...",ESCAMBIA,UNSURVEYED,30.74408,-86.562079
1,"POLYGON Z ((-9061671.5384 3555608.0978 0,-9061...",DUVAL,HANNA PARK,30.370955,-81.402843
2,"POLYGON Z ((-9054509.0537 3514807.6314 0,-9054...",ST JOHNS,GUANA RIVER SP,29.91218,-81.40989
3,"POLYGON Z ((-9668169.2045 3552697.6667 0,-9668...",ESCAMBIA,UNSURVEYED,30.74408,-86.562079
4,"POLYGON Z ((-9597884.3653 3547578.4878 0,-9597...",WALTON,WALTON COUNTY BCHS,26.677051,-80.052446
...,...,...,...,...,...
297,"POLYGON Z ((-9039717.3304 2937286.6217 0,-9039...",MONROE,ENP (HIGHLAND BEAC,25.55731,-80.91705
298,"POLYGON Z ((-9039593.2653 2937286.6434 0,-9039...",MONROE,ENP (HIGHLAND BEAC,25.55731,-80.91705
299,"POLYGON Z ((-9015596.0256 2844170.7527 0,-9015...",MONROE,LITTLE CRAWL KEY,24.742867,-80.983267
300,"POLYGON Z ((-9016208.8296 2843020.5594 0,-9016...",MONROE,FAT DEER KEY,24.73875,-81.00202


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299 entries, 0 to 301
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   WKT        299 non-null    object
 1   COUNTY     299 non-null    object
 2   NAME       299 non-null    object
 3   latitude   299 non-null    object
 4   longitude  299 non-null    object
dtypes: object(5)
memory usage: 22.1+ KB


In [36]:
grouped = df.groupby(['COUNTY', 'NAME']).size().reset_index(name='count')
filtered = grouped[grouped['count'] > 1]

# Create a new DataFrame with the filtered beaches
filtered_df = pd.merge(filtered, df, on=['COUNTY', 'NAME'], how='inner')


I also noticed some beaches sharing the same name in the same county, so we are gonna drop those and keep just one for each county since it will create the same entries of location with our geolocator

In [34]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   COUNTY     192 non-null    object
 1   NAME       192 non-null    object
 2   count      192 non-null    int64 
 3   WKT        192 non-null    object
 4   latitude   192 non-null    object
 5   longitude  192 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.1+ KB


In [35]:
filtered_df

,COUNTY,NAME,count,WKT,latitude,longitude
0,BAY,PANAMA CITY BCH,4,"POLYGON Z ((-9552376.2914 3526445.1693 0,-9552...",30.176591,-85.805386
1,BAY,PANAMA CITY BCH,4,"POLYGON Z ((-9570887.6124 3537280.2567 0,-9570...",30.176591,-85.805386
2,BAY,PANAMA CITY BCH,4,"POLYGON Z ((-9563378.7251 3533366.266 0,-95633...",30.176591,-85.805386
3,BAY,PANAMA CITY BCH,4,"POLYGON Z ((-9545646.8137 3520953.5789 0,-9545...",30.176591,-85.805386
4,BAY,ST ANDREWS SRA,3,"POLYGON Z ((-9544203.6352 3519628.9607 0,-9544...",30.23765,-85.63262
...,...,...,...,...,...,...
187,WALTON,WALTON COUNTY BCHS,9,"POLYGON Z ((-9598859.6815 3547890.1001 0,-9598...",26.677051,-80.052446
188,WALTON,WALTON COUNTY BCHS,9,"POLYGON Z ((-9594178.0695 3546344.4411 0,-9594...",26.677051,-80.052446
189,WALTON,WALTON COUNTY BCHS,9,"POLYGON Z ((-9587858.699 3544314.4707 0,-95877...",26.677051,-80.052446
190,WALTON,WALTON COUNTY BCHS,9,"POLYGON Z ((-9586984.1101 3543917.4127 0,-9587...",26.677051,-80.052446


# Model

In [17]:
starting_location = get_coordinates('Miami', '1642 SW 8th St')

Test location. Soon to be filled with the txt value.

In [32]:
df

,WKT,COUNTY,NAME,latitude,longitude
0,"POLYGON Z ((-9698711.156 3546590.3287 0,-96987...",ESCAMBIA,UNSURVEYED,30.74408,-86.562079
1,"POLYGON Z ((-9061671.5384 3555608.0978 0,-9061...",DUVAL,HANNA PARK,30.370955,-81.402843
2,"POLYGON Z ((-9054509.0537 3514807.6314 0,-9054...",ST JOHNS,GUANA RIVER SP,29.91218,-81.40989
3,"POLYGON Z ((-9668169.2045 3552697.6667 0,-9668...",ESCAMBIA,UNSURVEYED,30.74408,-86.562079
4,"POLYGON Z ((-9597884.3653 3547578.4878 0,-9597...",WALTON,WALTON COUNTY BCHS,26.677051,-80.052446
...,...,...,...,...,...
297,"POLYGON Z ((-9039717.3304 2937286.6217 0,-9039...",MONROE,ENP (HIGHLAND BEAC,25.55731,-80.91705
298,"POLYGON Z ((-9039593.2653 2937286.6434 0,-9039...",MONROE,ENP (HIGHLAND BEAC,25.55731,-80.91705
299,"POLYGON Z ((-9015596.0256 2844170.7527 0,-9015...",MONROE,LITTLE CRAWL KEY,24.742867,-80.983267
300,"POLYGON Z ((-9016208.8296 2843020.5594 0,-9016...",MONROE,FAT DEER KEY,24.73875,-81.00202


In [31]:
beaches_df = df.append(starting_location, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import numpy as np

def manhattan_distance(lat1, lon1, lat2, lon2):
    return abs(lat2 - lat1) + abs(lon2 - lon1)

# Calculate the distance matrix
n = len(beaches_df)
dist_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        if i != j:
            dist_matrix[i, j] = manhattan_distance(beaches_df.loc[i, 'latitude'], beaches_df.loc[i, 'longitude'],
                                                   beaches_df.loc[j, 'latitude'], beaches_df.loc[j, 'longitude'])
